# Reverse Linked List 2

Given the `head` of a singly linked list and two integers `left` and `right` where `left <= right`, reverse the nodes of the list from position left to position right, and return the reversed list. Note that `left` and `right` are indexed starting at 1, not 0.

[Leetcode Problem](https://leetcode.com/problems/reverse-linked-list-ii/description/)

### Example 1
Input: head = [1,2,3,4,5], left = 2, right = 4\
Output: [1,4,3,2,5]
### Example 2
Input: head = [5], left = 1, right = 1\
Output: [5]

### Constraints
* The number of nodes in the list is n.
* 1 <= n <= 500
* -500 <= Node.val <= 500
* 1 <= left <= right <= n

## Approach 1: Recursion
We can use recursion to help break up the steps I need to reverse the segment of the linked list. First, we want to iterate through the list until we get to the segment we want to reverse. We'll want to track with pointers multiple parts: the head right before the start of the segment, the start of the segment, the tail of the segment, and the node right after that segment. Using those parts, we can isolate the segment we want to reverse by disconnecting it from the rest of the list. Then, we perform the reversal on only that segment. And finally, we reconnect that segment to the rest of the list.

* Base Cases:
    1. head == None: return head (None)
    1. head.next = None: return head because there's nothing to reverse (only one item in list)
    1. left == right: return head (original list)
* Recursive Cases:
    1. left == 1: this means head is at the segment of the list that needs to be reversed.
    1. left != 1: this means we still haven't reach the segment to reverse yet
* State Change:
    1. We'll want to selectively reverse the rest of the list after `head`, track the end of the reversed segment with `tail` pointer, then reconnect `tail.next` to `head` and `head.next` to the rest of the list 
        * key insight: we can assign `tail` to `head.next` before we selectively reverse the rest of the list. That way, `tail` will automatically point to the end of the reversed segment without us having to iterate through the list again.    
    1. Since we haven't reached a segment to reverse yet, we'll want to iterate further into the list without modifying the list until we reach the segment
        * `head.next = reverseBetween(head.next, left-1, right-1)`
            * gets us closer to one of the base cases by inputing `head.next` and decrementing `left` and `right`

## Complexity Analysis
* Time Complexity: $O(N)$
    * In each layer of the stack, we do a constant number of operations, so each layer is O(1)
    * the stack depth is equal to the length of the list because worst case we'd need to iterate through the entire list to reach the base case, so O(N)
* Space Complexity: O(N)
    * The recursion stack depth is equal to the length of the linked list
* Code Complexity: Pretty decent
    * McCabe Cyclometric Complexity: 5 (Pretty good)
    * the base cases make up 3 different flows
    * Recursion splits the cases for dealing with different parts of the list into 2 different flows based on the `left == 1` condition
    * Overall, this is a pretty clean and straightforward way to split the problem into discrete parts with the tradeoff being space complexity and time complexity



In [ ]:
from typing import Optional


class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

def reverseBetween(self, head: Optional[ListNode], left: int, right: int) -> Optional[ListNode]:
    if not head or not head.next or left == right:
        return head
    if left == 1:
        tail = head.next
        nextNode = self.reverseBetween(head.next, left, right - 1)
        after_tail = tail.next
        tail.next = head
        head.next = after_tail
        head = nextNode
    else:
        head.next = reverseBetween(head.next, left-1, right-1)
    return head

## Approach 1: Expand on the "Reverse Linked List" Solution with more pointers

The `curr` pointer will be used to iterate through the entire linked list and the `prev` pointer allow us to track the previous node in each iteration. As we are iterating through the linked list, we need to track our position with the `pos` variable, so we can compare our current `pos` to `left` and `right`. 

We want to isolate the portion of the list we want to reverse, then reintegrate it with the rest of the list. To track the portions of the list we'd like to splice and then reconnect, we'll need another 4 pointers:
* `switch_head`: the head of the reversed portion of the list.
* `before_switch`: the node right before the reversed portion of the list, and will need to be reconnected to `switch_head` at the end
* `after_switch`: the node right after the reversed portion of the list
* `switch_end`: the tail of the reversed portion of the list, and will need to be reconnected with `after_switch`

We also have a toggle variable `reset_head` to determine whether we need to change the `head` of the list. This will be True if left == 1, because the node at `head` is part of the reversed portion of the list and will change positions.

We will iterate through the linked list with `curr` and `prev` normally and increment `pos` each time, but will perform additional operations under the following conditions:
* `pos < left`: update `before_switch` to equal `curr`
* `pos == left`: update `switch_end` to equal `curr`
* `pos > left and pos <= right+1`: `prev` is now in the reversed portion of the list. 
    * We now need to set `prev.next` to `switch_head` and `switch_head` to `prev` using round robin.
    * We also need to set `after_switch` to `curr`

After iterating through the entire list, our 4 pointers are assigned to the correct nodes. We now have to reassemble our list.

But also, we have to deal with a couple of edge cases:
1. `pos == right`: if the last element of the list is part of the reversed portion, then we still need to do one more pointer switching operation on the `prev` node and update `switch_head` to be the last element. In this case, we also shouldn't connect `switch_end` to `after_switch` because that will likely cause a cycle in some cases. 
1.  `pos != right and switch_end != after_switch`: in this case, we need to connect `switch_end` node to `after_switch` node
1. `reset_head is True`: we'll need to reset `head` to point to `switch_head`
1. `not reset_head and before_switch != switch_head`: if we are not reseting `head` to `switch_head`, then we'll need to connect `before_switch` node to `switch_head`  

### Complexity
* Time Complexity: O(N)
    * we only do one pass through the entire linked list of length N
* Space Complexity: O(1)
    * we store a constant number of pointers and other variables
* Code Complexity: Ok, but not good for interviews
    * McCabe Cyclomatic Complexity: 9 (good, but on the higher side for a simple procedure)
    * there are 6 different pointers to track (not counting `temp`) which can easily get confusing, especially as I'm reassigning them based on specific conditions as I iterate through the entire list
    * After iterating through the list, I have two normal cases determining how I reconnect nodes referenced by specific pointers. But I also have two edge cases I have to consider. It's not straightforward to set up the conditionals to capture the edge cases, while managing the normal cases. Getting the conditionals wrong can easily lead to forming cycles in the linked list, and causing memory errors.
    * Overall, this isn't a good solution for interviews nor on-the-job scenarios. It's easy to mess up and forget important details such as edge cases. It's hard to troubleshoot. And in the end, it's hard to explain how and why it works. 

In [ ]:
from typing import Optional
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

def reverseBetween(head: Optional[ListNode], left: int, right: int) -> Optional[ListNode]:
    pos = 0
    prev, curr = None, head
    
    switch_head = None
    before_switch = None
    switch_end = None
    after_switch = None
    reset_head = True if left == 1 else False

    while curr:
        pos += 1
        if pos < left:
            before_switch = curr
        elif pos == left:
            switch_end = curr
        elif pos > left and pos <= right+1:
            switch_now = prev
            switch_now.next = switch_head
            switch_head = switch_now 
            after_switch = curr
        temp = curr
        curr = curr.next
        prev = temp
    
    if pos == right:
        prev.next = switch_head
        switch_head = prev
    elif switch_end != after_switch:
        switch_end.next = after_switch
    if reset_head:
        head = switch_head
    elif before_switch != switch_head:
        before_switch.next = switch_head
    return head